This notebook is part of the [Machine Learning class](https://github.com/erachelson/MLclass) by [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en).

This notebook is joint work by Luca Mossina, Erwan Lecarpentier and Emmanuel Rachelson.

License: CC-BY-SA-NC.

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">SVMs (practice session)</div>

# Introduction

TODO

Content:
1. Linear SVMs
2. SVM with kernels
3. Application on TODO data set

# 1. Linear SVMs

TODO little intro

<div class="alert alert-warning">**Question 1.1**<br>
What is the objective of SVMs? On which principle is it built? Give a geometrical interpretation.
</div>

Your answer :

TODO Introduce the notations and provide optim problem for SVMs with **C = number of missclassified points**

TODO Say that we now import some artificial data in order to train an SVM classifier.

In [4]:
# TODO Code to import some toy data
# TODO Print the data

TODO Say to use scikit-learn to train an SVM classifier on those data

In [5]:
# TODO Uncomplete code that displays the data but needs to be filled

TODO Introduce cross validation
Now let us perform cross valdiation in order to select the best value of C -> give a range of values for C.

In [ ]:
# TODO Uncomplete code for cross validation

<div class="alert alert-warning">**Question 1.2**<br>
What is the best value of C among the proposed ones?
</div>

Your answer :

TODO introduce method to put C directly in the optimization problem

TODO Suggest to do it like this now

In [9]:
# TODO Uncomplete code for SVM with C directly in the optim pb

<div class="alert alert-warning">**Question 1.3**<br>
What is the best method?
</div>

# 2. SVMs with kernels

TODO little intro

# 3. Application

TODO little intro